In [1]:
import torch
import pandas as pd

import matplotlib.pyplot as plt

/Users/andrewt/miniforge3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import train as tr
import dataset as ds
import models as ml

# Load Test Dataset

In [3]:
mt_delta_series = pd.read_pickle('./data/pkl/mt_dataset_window_1024_stride_1024_crs_3857__.traj_delta_windows.pickle')
mt_ix_series = pd.read_pickle('./data/pkl/mt_dataset_train_dev_test_indices.pkl') 

In [4]:
mt_delta_series_train, mt_delta_series_test = mt_delta_series.iloc[mt_ix_series['train']].copy(), mt_delta_series.iloc[mt_ix_series['test']].copy() 

# Load Models

In [5]:
# VRF (share-nothing)
vrf_sn_cml = torch.load('./data/pth/lstm_1_350_fc_150_share_all_window_1024_stride_1024_crs_3857___batchsize_1__mt_dataset.pth', map_location=torch.device('cpu'))
print(vrf_sn_cml['scaler'].mean_, vrf_sn_cml['scaler'].scale_, sep='\t')

test_set = ds.VRFDataset(mt_delta_series_test, scaler=vrf_sn_cml['scaler'])
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, collate_fn=test_set.pad_collate)

vrf_sn_cml_model = ml.VesselRouteForecasting(
    hidden_size=350, fc_layers=[150,], scale=dict(
        mu=torch.tensor(vrf_sn_cml['scaler'].mean_[:2]), 
        sigma=torch.tensor(vrf_sn_cml['scaler'].scale_[:2])
    )
)
vrf_sn_cml_model.load_state_dict(vrf_sn_cml['model_state_dict'])
vrf_sn_cml_model.eval()

tr.evaluate_model(vrf_sn_cml_model, torch.device('cpu'), criterion=tr.RMSELoss(eps=1e-4), test_loader=test_loader)

/Users/andrewt/miniforge3/envs/torch/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[ -0.96859954  14.77653149 172.8909031  173.00816983]	[808.73788616 787.17245178 104.25695548 104.38338508]
self.eps=0.0001


Loss: 164.36008 |  Accuracy: 232.44022 | 148.99826; 284.90671; 520.33667; 785.26244; 1224.14243; 2077.23365; 1378.18813 m


(tensor(164.3601), 232.44021906810704)

In [6]:
# VRF (share-data)
vrf_sd_cml = torch.load('./data/pth/lstm_1_350_fc_150_share_all_window_1024_stride_1024_crs_3857___batchsize_1__share_all.pth', map_location=torch.device('cpu'))
print(vrf_sd_cml['scaler'].mean_, vrf_sd_cml['scaler'].scale_, sep='\t')

test_set = ds.VRFDataset(mt_delta_series_test, scaler=vrf_sd_cml['scaler'])
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, collate_fn=test_set.pad_collate)


vrf_sd_cml_model = ml.VesselRouteForecasting(
    hidden_size=350, fc_layers=[150,], scale=dict(
        mu=torch.tensor(vrf_sd_cml['scaler'].mean_[:2]), 
        sigma=torch.tensor(vrf_sd_cml['scaler'].scale_[:2])
    )
)
vrf_sd_cml_model.load_state_dict(vrf_sd_cml['model_state_dict'])
vrf_sd_cml_model.eval()

tr.evaluate_model(vrf_sd_cml_model, torch.device('cpu'), criterion=tr.RMSELoss(eps=1e-4), test_loader=test_loader)

/Users/andrewt/miniforge3/envs/torch/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[ 0.09815655  0.22542278 10.77163869 10.71747927]	[78.86009587 89.01867555 40.11217641 39.27786561]
self.eps=0.0001


Loss: 288.00873 |  Accuracy: 407.30584 | 258.72935; 511.18140; 1009.59893; 1458.71494; 1764.51012; 3006.27356; 4735.78398 m


(tensor(288.0087), 407.3058352127069)

In [5]:
# FedVRF
vrf_fl = torch.load('./data/pth/fl/lstm_1_350_fc_150_window_1024_stride_1024_crs_3857___.flwr_global_epoch170.pth', map_location=torch.device('cpu'))

train_set = ds.VRFDataset(mt_delta_series_train)
test_set = ds.VRFDataset(mt_delta_series_test, scaler=train_set.scaler)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, collate_fn=test_set.pad_collate)

fed_vrf = ml.VesselRouteForecasting(
    hidden_size=350, fc_layers=[150,], scale=dict(
        mu=torch.tensor(train_set.scaler.mean_[:2]), 
        sigma=torch.tensor(train_set.scaler.scale_[:2])
    )
)
fed_vrf.load_state_dict(vrf_fl['model_state_dict'])
fed_vrf.eval()

tr.evaluate_model(fed_vrf, torch.device('cpu'), criterion=tr.RMSELoss(eps=1e-4), test_loader=test_loader)

self.eps=0.0001


Loss: 1170.01111 |  Accuracy: 1654.64543 | 903.18347; 2917.02447; 4924.00062; 5909.73920; 7437.49085; 12748.15952; 9870.93581 m


(tensor(1170.0111), 1654.6454311191221)

In [5]:
# VRF (PerFL)
vrf_perfl = torch.load('./data/pth/perfl/lstm_1_350_fc_150_window_1024_stride_1024_crs_3857___.flwr_global_epoch170.pth', map_location=torch.device('cpu'))

train_set = ds.VRFDataset(mt_delta_series_train)
test_set = ds.VRFDataset(mt_delta_series_test, scaler=train_set.scaler)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, collate_fn=test_set.pad_collate)

fed_vrf = ml.VesselRouteForecasting(
    hidden_size=350, fc_layers=[150,], scale=dict(
        mu=torch.tensor(train_set.scaler.mean_[:2]), 
        sigma=torch.tensor(train_set.scaler.scale_[:2])
    )
)
fed_vrf.load_state_dict(vrf_perfl['model_state_dict'])
fed_vrf.eval()

tr.evaluate_model(fed_vrf, torch.device('cpu'), criterion=tr.RMSELoss(eps=1e-4), test_loader=test_loader)

self.eps=0.0001


Loss: 1180.60608 |  Accuracy: 1669.62919 | 947.07026; 2829.92602; 4866.28962; 6042.88501; 7534.40436; 11548.54689; 10034.59894 m


(tensor(1180.6061), 1669.629193045298)